#  Import

In [1]:
print("hello")

hello


In [2]:
import os
import sys

import numpy as  np
from lwm_model import lwm  # 클래스 import
import torch
import torch.nn.functional as F
from torch.utils.data import Dataset as TorchDataset, DataLoader, Subset
import time
import torch.nn as nn

from PIL import Image
import numpy as np

from deepverse import ParameterManager
from deepverse.scenario import ScenarioManager
from deepverse import Dataset

from deepverse.visualizers import ImageVisualizer, LidarVisualizer

# Settings

In [3]:
# Scenes 2000
## Subcarriers 64

scenarios_name = "DT31"
config_path = f"scenarios/{scenarios_name}/param/config.m"
param_manager = ParameterManager(config_path)

params = param_manager.get_params()

param_manager.params["scenes"] =list(range(100))
param_manager.params["comm"]["OFDM"]["selected_subcarriers"] = list(range(64))


# Generate a dataset

In [4]:
dataset = Dataset(param_manager)

Generating camera dataset: ⏳ In progress
Generating camera dataset: ✅ Completed (0.01s)
Generating LiDAR dataset: ⏳ In progress
Generating LiDAR dataset: ✅ Completed (0.00s)
Generating mobility dataset: ⏳ In progress
Generating mobility dataset: ✅ Completed (0.00s)
Generating comm dataset: ⏳ In progress


Generating comm dataset: ✅ Completed (4.99s)
Generating radar dataset: ⏳ In progress


Generating radar dataset: ✅ Completed (222.58s)


# location dataset 
 지금 실험에서는 안쓰임

In [5]:
# comm = dataset.comm_dataset
# location =  comm

# location = [
#     {
#         "bs_loc": d["bs_loc"],                      # (3,)
#         "ue_loc": np.asarray(d["ue_loc"]).squeeze() # (3,)  (원래 (1,3)이면 squeeze)
#     }
#     for row in comm.data      # row: [dict] 형태
#     for d in row              # d: dict
# ]

# print(ue_location)  #)

# communication  dataset

In [6]:
# UE 정보
comm = dataset.comm_dataset
ch = comm.data[0][0]['ue'][0]
print(ch.coeffs.shape)  

(1, 16, 64)


# preprocessing

In [7]:
def get_coeffs_from_frame(frame, ue_idx=0):
    ue_obj = frame["ue"]

    # 케이스1) list/tuple이면 ue_idx로 선택
    if isinstance(ue_obj, (list, tuple)):
        ch_obj = ue_obj[ue_idx]
    else:
        # 케이스2) 단일 OFDMChannel이면 그대로 사용
        ch_obj = ue_obj

    # coeffs는 dict key가 아니라 attribute일 확률이 매우 큼
    if hasattr(ch_obj, "coeffs"):
        return ch_obj.coeffs

    # 혹시 dict라면 마지막 보험
    if isinstance(ch_obj, dict) and "coeffs" in ch_obj:
        return ch_obj["coeffs"]

    raise TypeError(f"Cannot get coeffs. ue type={type(ue_obj)}, ch type={type(ch_obj)}")


In [ ]:
def get_train_min_max_realimag(frames, train_idx, us_idx=0):

    rmin, rmax =  float('inf'), float('-inf')
    imin, imax =  float('inf'), float('-inf')

    print("Calculating min/max over training set...")

    for t  in train_idx:
        frame  = frames[t]
        coeffs  = get_coeffs_from_frame(frame, us_idx)  # (N_subcarriers, )

        rmin = min(rmin, float(coeffs.real.min()))
        rmax = max(rmax, float(coeffs.real.max()))
        imin = min(imin, float(coeffs.imag.min()))
        imax = max(imax, float(coeffs.imag.max()))

    print(f"Done. rmin={rmin}, rmax={rmax}, imin={imin}, imax={imax}")
    return (rmin, rmax), (imin, imax)

In [9]:
def preprocess_channel_coeffs_minmax(coeffs_np, r_min, r_max, i_min, i_max, device="cuda", eps=1e-12):
    # Convert Numpy to Tensor
    coeffs = torch.from_numpy(coeffs_np).to(torch.complex64)
    
    r = coeffs.real
    i = coeffs.imag
    
    # Min-Max Scaling [0, 1]
    # Add eps to denominator to prevent division by zero
    r_scaled = (r - r_min) / max(r_max - r_min, eps)
    i_scaled = (i - i_min) / max(i_max - i_min, eps)
    
    # Concat (Maintains shape like (..., 2*subcarriers))
    H = torch.cat([r_scaled, i_scaled], dim=-1).to(device)
    return H

In [10]:
# 사용예시
H = preprocess_channel_coeffs_minmax(ch.coeffs, r_min=-0.5, r_max=0.5, i_min=-0.5, i_max=0.5)
print(H.shape)  # (1, 16, 128) 64 subcar

torch.Size([1, 16, 128])


# Dataset 구현

In [11]:
def flatten_comm_frames(comm):
    frames = []
    for row in comm.data:
        for d in row:
            frames.append(d)
    return frames

class ChannelNextStepDatasetGPU(TorchDataset):
    def __init__(self, comm_frames, ue_idx=0, past_len=16, device="cuda",
                 r_min=0.0, r_max=1.0, i_min=0.0, i_max=1.0):
        self.comm_frames = comm_frames
        self.ue_idx = ue_idx
        self.past_len = past_len
        self.device = device

        self.r_min, self.r_max = r_min, r_max
        self.i_min, self.i_max = i_min, i_max

        self.N = len(self.comm_frames)
        self.valid_start = past_len - 1
        self.valid_end = self.N - 2

    def __len__(self):
        return self.valid_end - self.valid_start + 1

    def __getitem__(self, idx):
        t = self.valid_start + idx

        # 1) Channel Past
        ch_list = []
        for k in range(t - self.past_len + 1, t + 1):
            coeffs_np = get_coeffs_from_frame(self.comm_frames[k], ue_idx=self.ue_idx)
            h = preprocess_channel_coeffs_minmax(
                coeffs_np,
                self.r_min, self.r_max, self.i_min, self.i_max,
                device=self.device
            ).reshape(-1)
            ch_list.append(h)
        channel_past = torch.stack(ch_list, dim=0)  # (T, F_in)

        # 2) Target (next step)
        coeffs_np_next = get_coeffs_from_frame(self.comm_frames[t + 1], ue_idx=self.ue_idx)
        target = preprocess_channel_coeffs_minmax(
            coeffs_np_next,
            self.r_min, self.r_max, self.i_min, self.i_max,
            device=self.device
        ).reshape(-1)  # (F_out)

        return channel_past, target


# DataLoader 구현

In [12]:
comm_frames = flatten_comm_frames(dataset.comm_dataset)

ds = ChannelNextStepDatasetGPU(
    comm_frames=comm_frames,
    ue_idx=0,
    past_len=15,
    device="cuda"
)

loader = DataLoader(
    ds,
    batch_size=8,
    shuffle=True,
    num_workers=0,     
    pin_memory=False   # ✅ 의미 없음 (이미 GPU)
)

ch, y = next(iter(loader))
print(ch.shape,  y.shape)
print(ch.device, y.device)
    

torch.Size([8, 15, 2048]) torch.Size([8, 2048])
cuda:0 cuda:0


# Fine-tuning
data shape 맞추기 위해

In [13]:
class LWMForecasterNoEdit(nn.Module):
    """
    lwm 코드는 수정하지 않고, encoder(embedding+layers)만 직접 호출해서 forecasting
    Input : ch  (B,T,F_in)
    Output: yhat(B,F_out)
    """
    def __init__(
            self,      
            base_lwm: nn.Module,
            F_in: int,
            F_out: int,
            pool="last",
            freeze_backbone: bool = False,
            element_length: int = 16,
            d_model: int = 64
            ):
        super().__init__()
        self.base = base_lwm
        self.pool = pool

        n_dim = self.base.embedding.element_length  # = ELEMENT_LENGTH(16)

        self.in_proj = nn.Sequential(
            nn.Linear(F_in, 512),
            nn.GELU(),
            nn.Linear(512, 128),
            nn.GELU(),
            nn.Linear(128, n_dim),
        )
        self.head = nn.Linear(d_model, F_out)

    def forward(self, ch):
        x = self.in_proj(ch)          # (B,T,F_in) -> (B,T,16)

        # ✅ lwm.forward() 쓰지 않고 encoder만 사용
        out = self.base.embedding(x)  # (B,T,64)
        for layer in self.base.layers:
            out, _ = layer(out)       # (B,T,64)

        if self.pool == "last":
            z = out[:, -1, :]
        elif self.pool == "mean":
            z = out.mean(dim=1)
        else:
            raise ValueError(f"Unknown pool={self.pool}")

        return self.head(z)           # (B,F_out)


## NMSE(dB)

In [14]:
@torch.no_grad()
def nmse_db(yhat: torch.Tensor, y: torch.Tensor, eps: float = 1e-12) -> torch.Tensor:
    # yhat, y: (B,F)
    num = torch.sum((yhat - y) ** 2, dim=1)
    den = torch.sum(y ** 2, dim=1).clamp_min(eps)
    nmse = num / den
    return 10.0 * torch.log10(nmse.clamp_min(eps)).mean()


# Train/Val split

In [15]:
n = len(ds)
n_train = int(0.75 * n)
train_idx = list(range(0, n_train))
val_idx = list(range(n_train, n))

train_ts = [ds.valid_start + i for i in train_idx]

(real_min,  real_max), (imag_min, imag_max) = get_train_min_max_realimag(
    comm_frames, train_ts, us_idx=0
)

ds.r_min = real_min
ds.r_max = real_max
ds.i_min = imag_min
ds.i_max = imag_max

print("Dataset statistical values set in the dataset.")

train_ds = Subset(ds, train_idx)
val_ds   = Subset(ds, val_idx)

train_loader = DataLoader(train_ds, batch_size=32, shuffle=True,  num_workers=0)
val_loader   = DataLoader(val_ds,   batch_size=32, shuffle=False, num_workers=0)

# Verify
ch, y = next(iter(train_loader))
print("\n=== Data Check ===")
print(f"y stats | min: {y.min().item():.4f}, max: {y.max().item():.4f}")
print("If scaling worked correctly, values should be within [0, 1].")
print("ch shape:", ch.shape, "y shape:", y.shape)   # 추가 확인용

Calculating min/max over training set...
Done. rmin=-1.0915750361793117e-06, rmax=1.1047814353805472e-06, imin=-1.1138026455803887e-06, imax=1.1093295598083908e-06
Dataset statistical values set in the dataset.

=== Data Check ===
y stats | min: 0.0123, max: 0.9743
If scaling worked correctly, values should be within [0, 1].
ch shape: torch.Size([32, 15, 2048]) y shape: torch.Size([32, 2048])


In [16]:
len(val_loader)

1

# Model generate and also check

In [17]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("device:", device)

# 배치 하나로 F_in/F_out 자동 확정 
ch, y = next(iter(train_loader))
F_in  = ch.shape[-1]
F_out = y.shape[-1]
print("Detected:", "F_in=", F_in, "F_out=", F_out)
print("Batch devices:", ch.device, y.device)

# --- base LWM (lwm_model은 수정 안 함) ---
# 너가 lwm 클래스를 정의해둔 파일에서 import 되어 있어야 함
# from lwm_model import lwm
base = lwm(
    element_length=16,
    d_model=64,
    max_len=129,
    n_layers=12
).to(device)

# 또는 pretrained 쓰면:
# base = lwm.from_pretrained("model_weights.pth", device=device)

# --- forecasting wrapper (fine-tuning 쪽에서만) ---
model = LWMForecasterNoEdit(
    base_lwm=base,
    F_in=F_in,
    F_out=F_out,
    pool="last"
).to(device)

# sanity forward
model.eval()
with torch.no_grad():
    yhat = model(ch.to(device))
print("yhat:", yhat.shape, "y:", y.shape)


device: cuda
Detected: F_in= 2048 F_out= 2048
Batch devices: cuda:0 cuda:0
yhat: torch.Size([32, 2048]) y: torch.Size([32, 2048])


# Train/ Eval 함수 (AMP + grad clip)

In [18]:
def train_one_epoch(model, loader, optimizer, device, use_amp=True, grad_clip=1.0):
    model.train()
    scaler = torch.cuda.amp.GradScaler(enabled=use_amp)

    total_loss = 0.0
    total_nmse = 0.0
    n = 0

    for ch, y in loader:
        # Dataset이 이미 cuda 텐서를 반환하더라도 안전하게 유지
        ch = ch.to(device, non_blocking=True)
        y  = y.to(device, non_blocking=True)

        optimizer.zero_grad(set_to_none=True)

        with torch.cuda.amp.autocast(enabled=use_amp):
            yhat = model(ch)
            loss = F.mse_loss(yhat, y)

        scaler.scale(loss).backward()

        if grad_clip is not None and grad_clip > 0:
            scaler.unscale_(optimizer)
            nn.utils.clip_grad_norm_(model.parameters(), grad_clip)

        scaler.step(optimizer)
        scaler.update()

        total_loss += loss.item()
        total_nmse += nmse_db(yhat.detach(), y).item()
        n += 1

    return total_loss / max(n, 1), total_nmse / max(n, 1)


@torch.no_grad()
@torch.no_grad()
def evaluate(model, loader, device):
    model.eval()

    total_loss = 0.0
    total_nmse = 0.0
    n = 0

    for ch, y in loader:  
        ch = ch.to(device, non_blocking=True)
        y  = y.to(device, non_blocking=True)

        yhat = model(ch)  
        loss = F.mse_loss(yhat, y)

        total_loss += loss.item()
        total_nmse += nmse_db(yhat, y).item()
        n += 1

    return total_loss / max(n, 1), total_nmse / max(n, 1)


#  Optiimizer  / Scheduler 설정

In [19]:
# requires_grad=True인 파라미터만 학습
trainable_params = [p for p in model.parameters() if p.requires_grad]
print("trainable params:", sum(p.numel() for p in trainable_params))

optimizer = torch.optim.AdamW(trainable_params, lr=1e-3, weight_decay=1e-4)

# (선택) cosine scheduler
epochs = 1000
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=epochs)


trainable params: 1853648


# 학습 루프 + checkpoint 저장

In [20]:
import time
import torch

best_val_loss = float("inf")
best_val_nmse = None
best_epoch = -1

t_train0 = time.time()  # ✅ 전체 학습 시작 시간

for epoch in range(1, epochs + 1):
    t0 = time.time()

    tr_loss, tr_nmse = train_one_epoch(
        model, train_loader, optimizer,
        device=device, use_amp=True, grad_clip=1.0
    )
    va_loss, va_nmse = evaluate(model, val_loader, device=device)

    scheduler.step()

    dt = time.time() - t0
    print(
        f"[{epoch:02d}/{epochs}] "
        f"train loss={tr_loss:.6f}, nmse(dB)={tr_nmse:.4f} | "
        f"val loss={va_loss:.6f}, nmse(dB)={va_nmse:.4f} | "
        f"{dt:.1f}s"
    )

    # ✅ best 체크포인트 저장 + best_nmse/epoch 기록
    if va_loss < best_val_loss:
        best_val_loss = va_loss
        best_val_nmse = va_nmse
        best_epoch = epoch

        torch.save(
            {
                "epoch": epoch,
                "model_state": model.state_dict(),
                "optimizer_state": optimizer.state_dict(),
                "F_in": F_in,
                "F_out": F_out,
                "best_val_loss": best_val_loss,   # (선택) 같이 저장
                "best_val_nmse": best_val_nmse,   # (선택) 같이 저장
            },
            "best_finetune.pt"
        )
        print("  ↳ saved best_finetune.pt")

# ✅ 루프가 끝난 뒤 요약 출력 (스샷 느낌)
total_sec = time.time() - t_train0
print("\n=== Done ===")
print(f"Best val loss     : {best_val_loss:.6f}")
print(f"Best val NMSE (dB): {best_val_nmse:.4f}")
print(f"Best epoch        : {best_epoch}")
print(f"Total train time  : {total_sec/60:.2f} min ({total_sec:.1f} sec)")


/tmp/ipykernel_880565/578535649.py:3: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = torch.cuda.amp.GradScaler(enabled=use_amp)
/tmp/ipykernel_880565/578535649.py:16: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(enabled=use_amp):


[01/1000] train loss=0.620560, nmse(dB)=3.3854 | val loss=0.428866, nmse(dB)=2.1903 | 2.0s
  ↳ saved best_finetune.pt
[02/1000] train loss=0.440337, nmse(dB)=1.8984 | val loss=0.308666, nmse(dB)=0.7614 | 0.2s
  ↳ saved best_finetune.pt
[03/1000] train loss=0.334929, nmse(dB)=0.7038 | val loss=0.235190, nmse(dB)=-0.4200 | 0.2s
  ↳ saved best_finetune.pt
[04/1000] train loss=0.274019, nmse(dB)=-0.1762 | val loss=0.193980, nmse(dB)=-1.2575 | 0.2s
  ↳ saved best_finetune.pt
[05/1000] train loss=0.240765, nmse(dB)=-0.7525 | val loss=0.169245, nmse(dB)=-1.8508 | 0.2s
  ↳ saved best_finetune.pt
[06/1000] train loss=0.219805, nmse(dB)=-1.1497 | val loss=0.145266, nmse(dB)=-2.5157 | 0.2s
  ↳ saved best_finetune.pt
[07/1000] train loss=0.194095, nmse(dB)=-1.6985 | val loss=0.118063, nmse(dB)=-3.4188 | 0.2s
  ↳ saved best_finetune.pt
[08/1000] train loss=0.164409, nmse(dB)=-2.4319 | val loss=0.092320, nmse(dB)=-4.4918 | 0.2s
  ↳ saved best_finetune.pt
[09/1000] train loss=0.135925, nmse(dB)=-3.26

In [21]:
# 채널-only: (ch, y)만 받음
ch, y = next(iter(train_loader))

print("y abs mean:", y.abs().mean().item())
print("y abs max :", y.abs().max().item())
print("y power   :", (y**2).mean().item())

with torch.no_grad():
    yhat = model(ch.to(device))   # img 제거

print("yhat abs mean:", yhat.abs().mean().item())
print("yhat abs max :", yhat.abs().max().item())
print("yhat power   :", (yhat**2).mean().item())


y abs mean: 0.5017839670181274
y abs max : 0.9797839522361755
y power   : 0.2924537658691406
yhat abs mean: 0.5001059770584106
yhat abs max : 1.0055065155029297
yhat power   : 0.28873661160469055


# 데이터 입력 및 형태

In [22]:
print("=== dataset sizes ===")
print("N(comm_frames):", len(comm_frames))
print("past_len      :", ds.past_len)
print("len(ds)       :", len(ds))
print("len(train_ds) :", len(train_ds))
print("len(val_ds)   :", len(val_ds))
print("len(train_loader):", len(train_loader))
print("len(val_loader)  :", len(val_loader))

print("\n=== one batch shapes ===")
ch, y = next(iter(train_loader))  # ✅ (ch, y)만 받기
print("ch :", tuple(ch.shape), " -> (B,T,F_in)")
print("y  :", tuple(y.shape), " -> (B,F_out)")

with torch.no_grad():
    yhat = model(ch.to(device))   # ✅ model(ch)만 호출

print("yhat:", tuple(yhat.shape), " -> (B,F_out)")
print("this forward predicted vectors:", yhat.shape[0], "(=B)")
print("each vector predicts elements:", yhat.shape[1], "(=F_out)")


=== dataset sizes ===
N(comm_frames): 100
past_len      : 15
len(ds)       : 85
len(train_ds) : 63
len(val_ds)   : 22
len(train_loader): 2
len(val_loader)  : 1

=== one batch shapes ===


ch : (32, 15, 2048)  -> (B,T,F_in)
y  : (32, 2048)  -> (B,F_out)
yhat: (32, 2048)  -> (B,F_out)
this forward predicted vectors: 32 (=B)
each vector predicts elements: 2048 (=F_out)
